In [ ]:
import chromedriver_autoinstaller
chromedriver_autoinstaller.install()

import requests
from bs4 import BeautifulSoup
import json
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time
import os
import csv

from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains


chrome_options = Options()
chrome_options.add_argument("--headless")   
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_argument("--window-size=1920,1080")  

driver = webdriver.Chrome(options=chrome_options)

In [ ]:
desktop_path = os.path.join(os.path.expanduser("~"), "Desktop\\TrendyolMilla\\Data\\Urls", "product_urls_final.csv")

df_product_links = pd.read_csv(desktop_path, encoding="utf-8-sig")

In [3]:
df_product_links.values

array([['https://www.trendyol-milla.com/trendyolmilla/siyah-duz-dokulu-fitted-vucuda-oturan-maxi-orme-elbise-twoaw25el00097-p-833107888'],
       ['https://www.trendyol-milla.com/trendyolmilla/siyah-kisa-kol-bodycone-vucuda-oturan-bisiklet-yaka-esnek-orme-maxi-kalem-elbise-twoss24el00097-p-778346763'],
       ['https://www.trendyol-milla.com/trendyolmilla/mavi-cep-detayli-mini-denim-gomlek-elbise-twoaw23el00643-p-288340209'],
       ...,
       ['https://www.trendyol-milla.com/trendyol-curve/siyah-dugme-detayli-yirtmacli-fitted-vucuda-oturan-midi-dokuma-elbise-tbbss24ah00214-p-819051941'],
       ['https://www.trendyol-milla.com/trendyol-curve/pembe-cicek-desenli-a-line-orme-buyuk-beden-elbise-tbbss25ah00127-p-888386896'],
       ['https://www.trendyol-milla.com/trendyolmilla/siyah-vucuda-oturan-payet-detayli-dokuma-mini-sik-elbise-tprss25el00204-p-885623096']],
      shape=(1973, 1), dtype=object)

In [ ]:
df_products = pd.DataFrame(columns=["Ürün Adı"])
df_products_non_comments = pd.DataFrame(columns=["Ürün Linki"])
df_comments = pd.DataFrame(columns=["Tarih","Boy","Kilo","Beden","Yorum","Ürün Adı"])

In [ ]:
def fetch_product_data(url_product):
    global df_products  

    response = requests.get(url_product)
    if response.status_code != 200:
        print("Sayfa yüklenemedi:", response.status_code)
        return df_products

    soup = BeautifulSoup(response.text, 'html.parser')


    product_name = soup.find('h1', class_='pr-new-br')
    product_name = product_name.text.strip() if product_name else "Bilinmiyor"


    attributes = {"Ürün Adı": product_name}


    detail_attr_items = soup.find_all('li', class_='detail-attr-item')
    for item in detail_attr_items:
        attribute_name = item.find('span', class_='attr-name')
        attribute_value = item.find('div', class_='attr-value-name-w')

        if attribute_name and attribute_value:
            attributes[attribute_name.text.strip()] = attribute_value.text.strip()


    new_row = pd.DataFrame([attributes])
    df_products = pd.concat([df_products, new_row], ignore_index=True)
    print("Ürün eklendi:", product_name)

    return df_products

In [ ]:
def fetch_comments_url(url_product):

    chrome_options = Options()
    chrome_options.add_argument("--headless")   
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    chrome_options.add_argument("--window-size=1920,1080")   


    driver = webdriver.Chrome(options=chrome_options)

    try:
        driver.get(url_product)


        time.sleep(5)  


        wait = WebDriverWait(driver, 10)  
        yorum_butonu = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'button.navigate-all-reviews-btn')))


        driver.execute_script("arguments[0].scrollIntoView(true);", yorum_butonu)


        actions = ActionChains(driver)
        actions.move_to_element(yorum_butonu).perform()


        driver.execute_script("arguments[0].click();", yorum_butonu)


        time.sleep(3)  
        url_comments = driver.current_url
        print(f"Yorum sayfasının URL'si: {url_comments}")

    except Exception as e:
        print("Buton bulunamadı veya tıklanamadı:", e)
        url_comments = None
    finally:

        driver.quit()

    return url_comments

In [ ]:
def fetch_comments(url_comments, product_name):

    chrome_options = Options()
    chrome_options.add_argument("--headless")
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    chrome_options.add_argument("--window-size=1920,1080")

    driver = webdriver.Chrome(options=chrome_options)
    driver.get(url_comments)

    yorumlar_listesi = []
    global df_comments  
    

    try:

        WebDriverWait(driver, 100).until(
            EC.presence_of_element_located((By.CLASS_NAME, "reviews-content"))
        )


        total_comments = int(driver.find_element(By.XPATH, "//div[@class='ps-ratings__counts']//div[contains(text(), 'Yorum')]/span[@class='ps-ratings__count-number']").text)


        last_scroll_height = driver.execute_script("return document.body.scrollHeight")   
        last_scroll_position = driver.execute_script("return window.pageYOffset")   

        while len(yorumlar_listesi) < total_comments:

            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")


            WebDriverWait(driver, 100).until(
                EC.presence_of_all_elements_located((By.CLASS_NAME, "comment"))
            )


            comments = driver.find_elements(By.CLASS_NAME, "comment")
            current_comment_count = len(comments)


            for comment in comments:
                try:

                    date = comment.find_element(By.XPATH, ".//div[@class='comment-info-item'][2]").text


                    try:
                        height = comment.find_element(By.XPATH, ".//div[@class='comment-info-item']//b[contains(text(), 'Boy')]/parent::span").text.strip()
                    except:
                        height = "Bilgi mevcut değil"

                    try:
                        weight = comment.find_element(By.XPATH, ".//div[@class='comment-info-item']//b[contains(text(), 'Kilo')]/parent::span").text.strip()
                    except:
                        weight = "Bilgi mevcut değil"

                    try:
                        size = comment.find_element(By.XPATH, ".//div[@class='comment-info-item']//b[contains(text(), 'Beden') or contains(text(), 'size')]/parent::span").text.strip()
                    except:
                        size = "Bilgi mevcut değil"

                    comment_text = comment.find_element(By.CLASS_NAME, "comment-text").text.strip()


                    if not comment_text:
                        comment_text = "Yorum yok"


                    yorumlar_listesi.append({
                        "Tarih": date,
                        "Boy": height,
                        "Kilo": weight,
                        "Beden": size,
                        "Yorum": comment_text,
                        "Ürün Adı": product_name
                    })

                except Exception as e:
                    print(f"Hata oluştu: {e}")

            print(f"Yorum sayısı: {len(yorumlar_listesi)} / {total_comments}")


            if len(yorumlar_listesi) >= total_comments:
                print("Tüm yorumlar alındı!")
                print("\n")

                break

    except Exception as e:
        print(f"Yorumlar yüklenirken hata oluştu: {e}")
    finally:
        driver.quit()


    df_new_comments = pd.DataFrame(yorumlar_listesi)


    df_comments = pd.concat([df_comments, df_new_comments], ignore_index=True)

    return df_comments


In [ ]:
product_path = os.path.join(os.path.expanduser("~"), "Desktop\\TrendyolMilla\\Data\\Products")
comment_path = os.path.join(os.path.expanduser("~"), "Desktop\\TrendyolMilla\\Data\\Comments")
non_comment_path = os.path.join(os.path.expanduser("~"), "Desktop\\TrendyolMilla\\Data\\Non_Comment_Products")

In [ ]:
prev_comments_len = 0

In [ ]:
for idx, url_product in enumerate(df_product_links["product_urls"], start=1) :   
    print(f"{idx} . Ürün Bilgileri alınmaktadır.")
    df_products = fetch_product_data(url_product)   
    url_comments = fetch_comments_url(url_product) 
    
    if url_comments is None:
        print(f"Yorum sayfasının URL'si alınamadı: {url_product}")
        df_products_non_comments.loc[len(df_products_non_comments)] = [url_product]
    else:
        product_name = df_products["Ürün Adı"].iloc[-1]   
        df_comments = fetch_comments(url_comments, product_name)  


    if idx % 10 == 0 or idx == len(df_product_links):
        file_name = f"products_{idx}.csv"
        file_path = os.path.join(product_path, file_name)
        df_products.to_csv(file_path, index=False)   
        print(f"{file_name} olarak kaydedildi.")


    if idx % 10 == 0 or idx == len(df_product_links):
        file_name = f"products_non_comments_{idx}.csv"
        file_path = os.path.join(non_comment_path, file_name)
        df_products_non_comments.to_csv(file_path, index=False)  
        print(f"{file_name} olarak kaydedildi.")



    if idx % 10 == 0 or idx == len(df_product_links):

        new_comments = df_comments.iloc[prev_comments_len:]   
        prev_comments_len = len(df_comments)  

        if not new_comments.empty:   
            file_name = f"comments_{idx}.csv"
            file_path = os.path.join(comment_path, file_name)
            new_comments.to_csv(file_path, index=False)   
            print(f"{file_name} olarak kaydedildi.") 
        
        print("\n")

1 . Ürün Bilgileri alınmaktadır.
Ürün eklendi: TRENDYOLMİLLA Siyah Düz Dokulu Fitted/Vücuda Oturan Maxi Örme Elbise TWOAW25EL00097
Yorum sayfasının URL'si: https://www.trendyol-milla.com/trendyolmilla/siyah-duz-dokulu-fitted-vucuda-oturan-maxi-orme-elbise-twoaw25el00097-p-833107888/yorumlar
Yorum sayısı: 30 / 155
Yorum sayısı: 90 / 155
Yorum sayısı: 180 / 155
Tüm yorumlar alındı!


2 . Ürün Bilgileri alınmaktadır.
Ürün eklendi: TRENDYOLMİLLA Siyah Kısa Kol Bodycone/Vücuda Oturan Bisiklet Yaka Esnek Örme Maxi Kalem Elbise TWOSS24EL00097
Yorum sayfasının URL'si: https://www.trendyol-milla.com/trendyolmilla/siyah-kisa-kol-bodycone-vucuda-oturan-bisiklet-yaka-esnek-orme-maxi-kalem-elbise-twoss24el00097-p-778346763/yorumlar
Yorum sayısı: 30 / 623
Yorum sayısı: 90 / 623
Yorum sayısı: 180 / 623
Yorum sayısı: 300 / 623
Yorum sayısı: 480 / 623
Yorum sayısı: 660 / 623
Tüm yorumlar alındı!


3 . Ürün Bilgileri alınmaktadır.
Ürün eklendi: TRENDYOLMİLLA Mavi Cep Detaylı Mini Denim Gömlek Elbise TWO